
*GROUP 19*
* DO Thi Duyen
* LE Ta Dang Khoa

In [ ]:

!pip3 install --user imbalanced-learn



In [ ]:

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import time
import pandas as pd
pd.set_option('display.max_columns', 130)

from collections import Counter

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import f1_score

from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks
from imblearn.over_sampling import SMOTE, BorderlineSMOTE

import zipfile
from io import BytesIO
from PIL import Image

from sklearn import preprocessing

import seaborn as sns
import matplotlib.pyplot as plt




<hr/>



<h2 id="1+2.-DATA-EXPLORATION-&amp;-PRE-PROCESSING"><strong>1+2. DATA EXPLORATION &amp; PRE-PROCESSING</strong><a class="anchor-link" href="#1+2.-DATA-EXPLORATION-&amp;-PRE-PROCESSING">¶</a></h2>



<h4 id="A.-Extract-Images-from-ZIP-file"><strong>A. Extract Images from ZIP file</strong><a class="anchor-link" href="#A.-Extract-Images-from-ZIP-file">¶</a></h4>


In [ ]:

def extract_zip_to_memory(input_zip):
    '''
    This function extracts the images stored inside the given zip file.
    It stores the result in a python dictionary.
    
    input_zip (string): path to the zip file
    
    returns (dict): {filename (string): image_file (bytes)}
    '''
    input_zip=zipfile.ZipFile(input_zip)
    return {name: BytesIO(input_zip.read(name)) for name in input_zip.namelist() if name.endswith('.jpg')}

start = time.time()
img_files = extract_zip_to_memory("/mnt/datasets/plankton/flowcam/imgs.zip")
print("Extraction time: ", time.time()-start)
print("Number of images: %d" % len(img_files))




<p><strong>Comment.</strong> The size of the dataset is pretty large (<strong>~240k</strong>), in comparison to the <em>Oregon State dataset</em> (<strong>~30K</strong>).</p>



<h4 id="B.-Take-a-look-at-the-Images"><strong>B. Take a look at the Images</strong><a class="anchor-link" href="#B.-Take-a-look-at-the-Images">¶</a></h4>


In [ ]:

def view_images(keys):
    """ Show images of planktons within the dataset,
        given the images' names.
    
    Parameters:
        keys (string): Name of the images
    Return:
        None.
    """
    for key in keys:
        # Read the Image
        img = imread(img_files[key])
    
        # Show the Image
        print(key)
        plt.imshow(img, cmap=cm.gray)
        plt.show()




<p><strong>FIRST</strong>, let's take look at the images themselves.</p>


In [ ]:

from skimage.io import imread, imshow
from pylab import cm
import random

# Randomly selects 5 images
img_keys = list(img_files.keys())
rand_names = random.sample(img_keys, 5)

# Show the images
view_images(rand_names)




<p><strong>NEXT</strong>, let's inspect the <strong>type &amp; depth</strong> of the images' pixels.</p>


In [ ]:

modes = [Image.open(img_files[k]).mode for k in img_files]
print(Counter(modes))




<p><strong>Comment.</strong></p>
<ol>
<li><p>Since all <strong>243,610</strong> images of this dataset is of mode L, we can conclude that our image-dataset are <strong>8-bit color pixels</strong> and <strong>greyscale</strong>.</p>
</li>
<li><p>After running several times this process of randomly showing images, we see that the size (dimensions) of the images are very <strong>different</strong>, each <strong>crops exactly</strong> to the plankton's object.</p>
</li>
<li><p>The implication of this is if one wants to use <em>CNN</em> on this dataset, then she has to resize all the images to a same size, a very <strong>time-consuming</strong> process.</p>
</li>
</ol>



<p><strong>FINALLY</strong>, we plot the distribution of images' sizes to see if we can get some more insights.</p>


In [ ]:

# Get the dimensions of the image
dims = [Image.open(img_files[k]).size for k in img_files]

# Get the ratios of the image
wh_ratios = [(w / h) for w, h in dims]
widths = [w for w, _ in dims]
heights = [h for _, h in dims]



In [ ]:

# Graph the distribution of width-height ratios
plt.figure()
r = sns.distplot(wh_ratios, bins=1000, kde=False)
r.set(xlim=(0, 5), title='width / height ratio')

# Graph the distribution of widths
plt.figure()
sns.distplot(widths).set_title('widths')

# Graph the distribution of heights
plt.figure()
sns.distplot(heights).set_title('heights')




<p><strong>COMMENT.</strong></p>
<ol>
<li><p>Looking at the <strong>width-height ratio</strong> graph, we can see that:</p>
<ul>
<li>the images are mostly square (<em>ratio = 1</em>), </li>
<li>a big amount of images are <em>horizontal rectangles</em> (<em>ratio &lt; 1 =&gt; width &lt; height</em>),</li>
<li>and the images rarely are <em>vettical rectangles</em> (<em>ratio &gt; 1 =&gt; width &gt; height</em>).</li>
</ul>
</li>
<li><p>Looking at the <strong>widths</strong> and <strong>heights</strong> graph, we can see that both share a very similar distribution, <em>left-skewed</em> with <em>modes around 100</em>.</p>
</li>
<li><p>The overall implication is that, in this dataset, <em>image-size</em> &amp; <em>image-shape</em> are <strong>very different</strong>. Therefore, scaling them to a same size and apply <em>CNN</em> will encode <strong>wrong (latent) signals</strong> in <em>a BIG proportion</em> of images. Unless we're very skillful at image processing to resolve this problem (which we surely aren't), we should avoid that <em>CNN</em> path.</p>
</li>
</ol>



<h4 id="C.-Analysing-the-Labels"><strong>C. Analysing the Labels</strong><a class="anchor-link" href="#C.-Analysing-the-Labels">¶</a></h4>



<p><strong>FIRST</strong>, we load the <strong>meta-data</strong> of the images.</p>


In [ ]:

meta = pd.read_csv('/mnt/datasets/plankton/flowcam/meta.csv')
meta.head(5)




<p><strong>COMMENT</strong>. A big suprise when we look at the labels of this dataset is that many of them aren't planktons in a common sense.</p>
<ul>
<li>For example, <em>detritus</em> is actually about dead bodies; <em>feces</em> is, of course, not a usual plankton; and <em>badfocus</em> is actually about images that are taken with a wrong focus.</li>
<li>When taking a deeper look at the <strong>lineage-column</strong> of this meta-data, one can see the full taxonomy that also includes _silks, <em>artefacts</em> and many types that aren't usual plankton.</li>
<li>We would speculate that if this data is representative, then it is <strong>highly unblanced</strong>, as <em>dead bodies</em> are overwhelmed in the sea, and <em>silk or plastic</em> cannot share the same proportion with real planktons.</li>
</ul>



<p><strong>NEXT</strong>, we build the <strong>targets</strong> dataframe, which includes <em>object-ids</em> and their corresponding <em>level-2 labels</em>.</p>


In [ ]:

targets_df = meta[['objid','level2']].drop_duplicates()
targets_df = targets_df[targets_df['level2'].isna()!=True]
targets_df['objid'] = targets_df['objid'].astype('int64')

targets_df.head(5)



In [ ]:

print("Number of records: %d" % targets_df.shape[0])
print("Number of classes: %d" % targets_df['level2'].nunique())




<p><strong>COMMENT</strong>. Here we build the <strong>targets</strong> set as followings:</p>
<ol>
<li>Only choose 2 columns <em>objid</em> (<strong>image-id</strong>) and <em>level2</em> (<strong>label</strong>) because they are all we need.</li>
<li>Remove all rows that doesn't have a label, as we cannot train with them.</li>
</ol>
<p>After this process, there are <strong>242,607</strong> records left. In comparison with <strong>243,610</strong> images in the dataset, this is smaller. Therefore, we need to make sure that all records in the <em>target-set</em> are included in the <em>image-dataset</em>. Since otherwise, we will train our models on objects that aren't existed.</p>



<p><strong>FINALLY</strong>, we plot the distribution of labels.</p>


In [ ]:

labels_freq = Counter(targets_df['level2'])
labels_freq_df = pd.DataFrame.from_dict(labels_freq, orient='index')\
                             .rename(columns={0: 'count'})\
                             .sort_values(by=['count'], ascending=False)

labels_freq_df.plot(kind='bar');




<p><strong>COMMENT.</strong> As speculated above, this dataset is highly imblanced.</p>
<ol>
<li><p>The biggest class has with 77812 instances (<em>detritus</em> or dead bodies), while the smallest class has only 8 instances (<em>Bacteriastrum</em>, a real plankton).</p>
</li>
<li><p>It is very difficult to get high performance if we leave this dataset imbalanced as it is. Therefore, <strong>resampling techniques</strong> are very needed, possible <strong>over-sampling</strong> for minor classes.</p>
</li>
</ol>



<h4 id="D.-Encode-the-Labels.-Check-if-object-ids-are-in-synced-with-image-ids."><strong>D. Encode the Labels. Check if object-ids are in synced with image-ids.</strong><a class="anchor-link" href="#D.-Encode-the-Labels.-Check-if-object-ids-are-in-synced-with-image-ids.">¶</a></h4>



<p><strong>FIRST</strong>, we encode the labels of our <strong>targets</strong> dataframe.</p>


In [ ]:

# Initialize the LabelEncoder
le = preprocessing.LabelEncoder()
le.fit(targets_df['level2'])

# Encode the labels, drop the original column
targets_df['target'] = le.transform(targets_df['level2'])
targets_df.drop(['level2'], axis=1, inplace=True)

targets_df.head(5)




<p><strong>NEXT</strong>, we ensure that all <em>object-ids</em> are belong the <em>image-ids</em>.</p>


In [ ]:

# Get the ID of all images
image_ids = [int(k.split('/')[1].split('.')[0]) for k in img_files]
image_ids_set = set(image_ids)



In [ ]:

# Check if all object-ids are belong to image-ids
object_ids_set = set(targets_df['objid'])
object_ids_set.difference(image_ids_set)




<p><strong>COMMENT</strong>. Since all <em>object-ids</em> refer to images in our image-dataset, we can proceed with <em>feature exploration</em>.</p>



<h4 id="E.-Explore-the-Features:"><strong>E. Explore the Features</strong>:<a class="anchor-link" href="#E.-Explore-the-Features:">¶</a></h4>



<p><strong><em>WHY WE USE FEATURES INSTEAD OF IMAGES?</em></strong></p>
<ol>
<li>First, as stated above, resizing images is not a good idea.</li>
<li>Second, but much more important, is that the features given are engineered with <strong>expert-knolwedge</strong>:<ul>
<li>If look for a partial of this dataset online, one will see that the features are constructed using an expertise process name <strong>ZooProcess</strong>.</li>
<li>This process computes <em>morphological features</em> in the <strong>native feature-set</strong>.</li>
<li>Then, they use the <strong>skimage</strong> library to recompute the <em>morphological features</em> using the ROIs produced by ZooProcess.</li>
<li>ROIs are <strong>regions of interested</strong>, which can only be pointed out by expert knowledge.</li>
</ul>
</li>
</ol>
<p>That's why we decided to use <em>features</em> instead (both of them, indeed).</p>



<p><strong>TO BEGIN</strong>, we investigate the <em>Native</em> feature-set, and see if there are anything we should do to preprocess the data.</p>


In [ ]:

# Load the Native Feature-set
features_native = pd.read_csv('/mnt/datasets/plankton/flowcam/features_native.csv.gz', compression='gzip', error_bad_lines=False)
features_native['objid'] = features_native['objid'].astype('int64')




<p>We find all columns that contains at least one <em>NaN</em>.</p>


In [ ]:

features_native.loc[:, features_native.isna().any()][:10]




<p><strong>COMMENT</strong>.</p>
<ul>
<li>The <em>NaN</em> are due to some problems with the records, not the inherent flaws of the features. Therefore, we need to fill in the missing values instead of dropping columns.</li>
<li>Next, let's find the rows that have the values at those columns equals <em>NaN</em>, to see if there are any patterns among them.</li>
</ul>


In [ ]:

# For the columns: 'perimareaexc', 'feretareaexc' & 'cdexc'
display(features_native[features_native['perimareaexc'].isnull()].loc[:, features_native.isna().any()][:10])

# For the columns: 'convarea_area', 'convarea_area' 'symetrieh_area', 'symetriev_area'
# 'nb1_area', 'nb2_area', 'nb3_area' & 'skeleton_area'
display(features_native[features_native['convarea_area'].isnull()].loc[:, features_native.isna().any()][:10])




<p><strong>COMMENT</strong>. The rows identified for inspection are:</p>
<ul>
<li>To fill in the missing values for <strong>perimareaexc, feretareaexc, cdexc</strong>, let's inspect <em>17, 22, 29</em> (select randomly).</li>
<li>To fill in the missing values for <strong>convarea_area, symetrieh_area, symetriev_area, nb1_area, nb2_area, nb3_area, skeleton_area</strong>, let's inspect <em>826, 842, 855</em> (select randomly).</li>
</ul>


In [ ]:

# Get the obj-ids of object at index = 17, 22, 29
focus_ids = features_native.loc[[17, 22, 29], ['objid']].objid.tolist()
focus_keys = ["imgs/{}.jpg".format(id) for id in focus_ids]

# View their images
view_images(focus_keys)



In [ ]:

# Get the obj-ids of object at index = 826, 842, 855
focus_ids = features_native.loc[[826, 842, 855], ['objid']].objid.tolist()
focus_keys = ["imgs/{}.jpg".format(id) for id in focus_ids]

# View their images
view_images(focus_keys)




<p><strong>COMMENT</strong>. In each case, we can that the images are very similar, showing a systemic <strong>evidence of absence</strong>, <em>NOT</em> random <strong>absence of evidence</strong>:</p>
<ul>
<li>Therefore, there must be a reason why the experts decided not to fill in the values. </li>
<li>So, we should fill in 0 instead of something else (such as <em>mode</em>, <em>mean</em>, <em>median</em>, etc.).</li>
</ul>


In [ ]:

# Fill all missing values with 0
features_native.fillna(0, inplace=True)




<p><strong>NEXT</strong>, we investigate the <em>skimage</em> feature-set, for the purpose of data-preprocessing.</p>


In [ ]:

# Load the skimage's feature-set
features_skimage = pd.read_csv('/mnt/datasets/plankton/flowcam/features_skimage.csv.gz', compression='gzip', error_bad_lines=False)
features_skimage['objid'] = features_skimage['objid'].astype('int64')



In [ ]:

# View ALL columns that contains NaN
features_skimage.loc[:10, features_skimage.isna().any()]




<p><strong>COMMENT</strong>. These <em>columns</em> are inherently flawed, as they contains mostly <em>NaN</em>, which makes sense because the features here are recomputed using <strong>skimage</strong> library, a very general library. So, it is normal that there are features cannot be computed. Therefore, we should remove all these <em>empty features</em>.</p>


In [ ]:

# Drop the 'flawed' columns
features_skimage.dropna(axis='columns', inplace=True)




<p><strong>NOW</strong>, we'll merge the 2 feature-sets with the targets, using <em>object-ids</em>.</p>


In [ ]:

data = features_native.merge(features_skimage, on='objid', how='inner').merge(targets_df, on='objid', how='inner')
print("Data after joining features and target: ", data.shape)
data.head(10)




<p><strong>FINALLY</strong>, we measure the <em>mutual-information</em> between the <strong>merged features</strong> and the <strong>targets</strong>.</p>


In [ ]:

from sklearn.feature_selection import mutual_info_classif

# Get the Merged-Features
merged_features = data.drop(columns=["objid", "target"])

# Get Mutual Information between 'target' and each 'feature'
# Then turn it into a DataFrame, and sort descendingly
mutual_info = mutual_info_classif(merged_features, data['target'])

mutual_info_df = pd.DataFrame(data=mutual_info, index=merged_features.columns, columns=['MI'])
mutual_info_df.sort_values(by='MI', inplace=True, ascending=False)




<p>To know how good these <em>mutual-information</em> are, we compare them to the entropy of the <em>targets</em>.</p>


In [ ]:

from scipy.stats import entropy

targets_freq = data['target'].value_counts().tolist()
targets_entropy = entropy(targets_freq)

print(targets_entropy)



In [ ]:

# Show the head & tail of mutual-information data-frame
display(mutual_info_df[:5])
display(mutual_info_df[-5:])

print("# of features: {}.".format(len(mutual_info_df)))
print("# of features that reduces uncertainty BY-MORE-THAN-10%: {}.".\
          format(len(mutual_info_df[mutual_info_df['MI'] > 0.1*targets_entropy])))




<p><strong>COMMENT:</strong></p>
<ul>
<li>First, 108/123 features is quite a good number, given that they are the features that reduces <strong>more-than-10% the uncertainty</strong> of the labels.</li>
<li>This probably confirms out intuition about <em>features engineered using expert-knowledge</em>:<ol>
<li>First, <strong>by definition</strong>, planktons are categorized by a multitude of <em>morphological features</em>.</li>
<li>Second, the features here are identified using <strong>regions of interests (ROIs)</strong> marked by experts.</li>
</ol>
</li>
</ul>
<p>Therefore, see almost all of them reduces <strong>uncertainty</strong> for a good amount is really confirming.</p>



<h4 id="F.-Split-Train-Validation-Test-sets"><strong>F. Split Train-Validation-Test sets</strong><a class="anchor-link" href="#F.-Split-Train-Validation-Test-sets">¶</a></h4>


In [ ]:

# Reverse the label-encoding, as we don't need it anymore
data['target'] = le.inverse_transform(data['target'])

# Split the training set, validation set and testing set with ratio as 0.5 : 0.25 : 0.25
X_train, X_test, y_train, y_test = train_test_split(data.drop(['objid', 'target'], axis=1), data['target'], test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

print("Number of classes of training set: ", y_train.nunique())
print("Number of classes of validation set: ", y_val.nunique())
print("Number of classes of testing set: ", y_test.nunique())




<p><strong>COMMENT.</strong></p>
<ul>
<li>Here we split the full set into <em>training set</em>, <em>validation set</em> and <em>testing set</em> with ratio 0.5, 0.25, 0.25 respectively. </li>
<li>Each splitted dataset <strong>keep the same number of classes</strong> as the full set (<strong>39 classes</strong>).</li>
</ul>



<hr/>
<h2 id="3.-SOLVING-IMBALANCE-DATASET"><strong>3. SOLVING IMBALANCE DATASET</strong><a class="anchor-link" href="#3.-SOLVING-IMBALANCE-DATASET">¶</a></h2>



<p><strong>MAIN IDEAS</strong>.</p>
<ol>
<li><p><em>Why we need to solve this?</em> Because otherwise, our trained-models will have low predictive power for the <strong>infrequent classes</strong> of the dataset.</p>
</li>
<li><p><em>How are we going to solve this?</em></p>
<ul>
<li>Since the dataset is very imblanced, we use both <strong>oversampling</strong> and <strong>undersampling</strong> techniques. </li>
<li>In addition to that, we use a cleaning method called <strong>TomekLinks</strong> to improve the resamplings.</li>
</ul>
</li>
<li><p><em>How do we know which resamplings are good?</em></p>
<ul>
<li>We use a <em>Baseline Model</em> and the <em>Validation Set</em> to measure the performances of our resamplings.</li>
<li>The metric we used are <em>f1-score</em> with <em>avg = 'macro'</em>.</li>
</ul>
</li>
</ol>


In [ ]:

# A helper function to display the score
def report_f1(y_test, y_pred):
    """ Return the well-displayed scores for evaluation.
    
    Parameters:
        y_test: the true labels
        y_pred: the predicted labels
    Return:
        result (DataFrame): the dataframe of scores
    """
    # Evaluate the performance using sklearn library
    report = classification_report(y_test, y_pred, output_dict=True)
    report_df = pd.DataFrame.from_dict(report).transpose()
    
    idx_first = ['macro avg']
    
    # Re-order to show f1 macro first
    df1 = report_df.ix[idx_first]
    idx_df2 = list(set(report_df.index)-set(idx_first + ['weighted avg', 'micro avg']))
    df2 = report_df.ix[idx_df2].sort_values(by = ['f1-score'], ascending=False)
    result = pd.concat([df1, df2]).transpose()
    
    return result



In [ ]:

# Define a baseline model to fit, predict and evaluate the input data sets and models
def baseline_model(X_train, y_train, X_test, y_test, model):
    """ Return the evaluation of given models and given inputs.
    
    Parameters:
        X_train: features for training
        y_train: true labels for training
        X_test: features for testing/validation
        y_test: true labels for testing/validation
        model: the learning model
    Return:
        A dictionary of evaluation
    """
    start = time.time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    running_time = time.time()-start
    report = report_f1(y_test, y_pred)
    print("Time execution: ", running_time)
    return {'model': model, 'y_pred': y_pred, 'running_time': running_time, 'report': report}




<h4 id="A.-The-Baseline-Model-&amp;--Feature-Importance"><strong>A. The Baseline Model &amp;  Feature Importance</strong><a class="anchor-link" href="#A.-The-Baseline-Model-&amp;--Feature-Importance">¶</a></h4>



<p><strong>FIRST</strong>, for the baseline-model, we'll use <em>RandomForestClassifier</em>.</p>


In [ ]:

rf = RandomForestClassifier(n_estimators=100, random_state=42)
baseline_rf = baseline_model(X_train, y_train, X_val, y_val, rf)
baseline_rf['report']




<p><strong>COMMENT-1.</strong></p>
<ul>
<li><strong>F1 avg macro</strong> for the baseline model is <strong>0.441259</strong></li>
<li><p>The high performances belong to the classes like <em>badfocus (artefact)</em>, <em>silks</em>, <em>detritus</em>, <em>Codonellopsis (Dictyocystidae)</em>, <em>Neoceratium</em>, <em>Thalassionema</em> and <em>rods</em>. The classes are major classes and/or having specific shapes of images.</p>
</li>
<li><p>The low performances belong to the minor classes which do not have enough information to classify.</p>
</li>
</ul>
<p><strong>COMMENT-2.</strong></p>
<ul>
<li>The highest score belongs to the class <em>badfocus (artefact)</em>. In the training set, this class has 4416 instances while the biggest class has 77812 instances. </li>
<li>So, <strong>more data does not mean more accuracy</strong>, we will consider this aspect when choose the number of instances for resampling data.</li>
</ul>



<p><strong>NEXT</strong>:</p>
<ul>
<li>As stated before, the features used in this dataset are somwhat based on <strong>expert knowledge</strong>, so it is not really necessary to do <em>feature-selection</em>. </li>
<li>However, here we take the baseline model as an opportunity to compute the importances of each feature to verify that conclusion.</li>
</ul>


In [ ]:

feature_importance = baseline_rf['model'].feature_importances_
plt.bar(range(0, len(feature_importance)), sorted(feature_importance, reverse=True));




<p><strong>COMMENT.</strong></p>
<p>The feature importances decrease gradually in this graph. There are only four features which have very low importances. Therefore, we keep all the features for learning models.</p>



<h4 id="B.-Resampling:-over-sampling,-under-sampling-&amp;-TomekLinks"><strong>B. Resampling: over-sampling, under-sampling &amp; TomekLinks</strong><a class="anchor-link" href="#B.-Resampling:-over-sampling,-under-sampling-&amp;-TomekLinks">¶</a></h4>


In [ ]:

# A helper funtion to resampling several classes
def resampling_multiclass(X_train, y_train, alg, classes):
    """ Resample the given classes while remainig the other classes
    Parameters:
        X_train: Features for training
        y_train: Labels for training
        alg: A method for resampling
        classes: The classes to resample
    Returns: 
        X_train_new: Features after resampling
        y_train_new: Labels after resampling
    """
    
    # Get row indices of given classes
    idx_g1 = y_train[y_train.isin(classes)].index
    # Get row indices of the remaining classes
    idx_g2 = y_train[~y_train.isin(classes)].index
    
    # Get new set for features according to idx_g1
    X_train_g1 = X_train.ix[idx_g1]
    y_train_g1 = y_train.ix[idx_g1]
    # Get new set for features according to idx_g2
    X_train_g2 = X_train.ix[idx_g2]
    y_train_g2 = y_train.ix[idx_g2]
    
    # Resample the new set of the given classes
    X_res, y_res = alg.fit_resample(X_train_g1, y_train_g1)
    
    # Concatenate the resampled sets and non-resample sets
    X_train_new = np.concatenate([X_res, X_train_g2])
    y_train_new = np.concatenate([y_res, y_train_g2])
    
    return X_train_new, y_train_new




<p><strong>FIRST</strong> is <strong>oversampling</strong> for <em>infrequent classes</em>.</p>
<ul>
<li><p>Here we use <em>SMOTE</em> (Synthetic Minority Oversampling Technique), a very popular oversampling method that was proposed to improve random oversampling.</p>
</li>
<li><p>About <em>SMOTE</em>: Rather than replicating the minority observations, SMOTE works by creating <strong>synthetic observations</strong> in by interpolation. The basic implementation of SMOTE will not make any distinction between easy and hard samples to be classified using the nearest neighbors rule.</p>
</li>
<li><p>We also use another SMOTE variants, <em>Borderline-SMOTE</em>. This variant detects which point to select in the border between two classes.</p>
</li>
</ul>


In [ ]:

sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = resampling_multiclass(X_train, y_train, sm, ['multiple (other)', 'badfocus (artefact)', 'silks', 'Copepoda', 'Thalassionema', 'rods', 'Codonellopsis (Dictyocystidae)', 'Protoperidinium', 'Tintinnidiidae', 'Rhizosolenids', 'Chaetoceros', 'artefact', 'pollen', 'Codonaria', 'chainlarge', 'Undellidae', 'Hemiaulus', 'egg (other)', 'Dinophysiales', 'Dictyocysta', 'Annelida', 'Stenosemella', 'Rhabdonella', 'Coscinodiscids', 'Retaria', 'Pleurosigma', 'Ceratocorys horrida', 'centric', 'Odontella (Mediophyceae)', 'Asterionellopsis', 'Cyttarocylis', 'Lithodesmioides', 'tempChaetoceros danicus', 'Xystonellidae', 'Bacteriastrum'])
baseline_model(X_train_sm, y_train_sm, X_val, y_val, rf)['report']



In [ ]:

bsm = BorderlineSMOTE(random_state=42)
X_train_bsm, y_train_bsm = resampling_multiclass(X_train, y_train, bsm, ['multiple (other)', 'badfocus (artefact)', 'silks', 'Copepoda', 'Thalassionema', 'rods', 'Codonellopsis (Dictyocystidae)', 'Protoperidinium', 'Tintinnidiidae', 'Rhizosolenids', 'Chaetoceros', 'artefact', 'pollen', 'Codonaria', 'chainlarge', 'Undellidae', 'Hemiaulus', 'egg (other)', 'Dinophysiales', 'Dictyocysta', 'Annelida', 'Stenosemella', 'Rhabdonella', 'Coscinodiscids', 'Retaria', 'Pleurosigma', 'Ceratocorys horrida', 'centric', 'Odontella (Mediophyceae)', 'Asterionellopsis', 'Cyttarocylis', 'Lithodesmioides', 'tempChaetoceros danicus', 'Xystonellidae', 'Bacteriastrum'])
baseline_model(X_train_bsm, y_train_bsm, X_val, y_val, rf)['report']




<p><strong>COMMENT.</strong></p>
<p>Based on the results of the baseline model, we varied the number of classes for oversampling and choose to oversample data for 33 minor classes. Each of them was oversampled up to 4416 instances. We choose this number for oversampling the minor classes because this number is not a too big but still can gives the highest f1 score (in the baseline model, <em>badfocus (artefact)</em> has 4416 instances in the training set and gets the highesh f1-score macro as 0.952245).</p>
<ol>
<li><p>Results when oversampling:</p>
<ul>
<li>Using SMOTE: <ul>
<li>F1-score macro: <strong>0.543803</strong></li>
<li>Running time: <strong>642 seconds</strong></li>
</ul>
</li>
<li>Using Borderline-SMOTE:<ul>
<li>F1-score macro: <strong>0.517850</strong></li>
<li>Running time: <strong>599 seconds</strong></li>
</ul>
</li>
</ul>
</li>
<li><p>Compare to the baseline model, the model after oversampling training data gives the better score (<strong>increase ~ 10.25%</strong>, from 54.38% to 44.13%). This is a <strong>significant improvement</strong>.</p>
</li>
<li><p>Compare to Borderline-SMOTE, SMOTE method runs a bit slower but gives the better score (54.38%). We prefer to improve the score, therfore, we choose SMOTE as the technique for oversampling in this project.</p>
</li>
<li><p>Further work: As can be seen from the two output tables above, each method SMOTE and Borderline-SMOTE may improve different classes. Therefore, we may combine them to oversample classes.</p>
</li>
</ol>



<p><strong>NEXT</strong> is <strong>under-sampling</strong>.</p>
<p><strong>The controlled under-sampling methods</strong>: we can specify the expected number of instances after undersampling.
After oversampling, we use RandomUnderSampler and NearMiss-1 to undersample the major classes.</p>
<ul>
<li><em>RandomUnderSampler</em>: it undersample the majority classes by randomly picking samples.</li>
<li><em>NearMiss-1</em>: it selects samples from the majority class for which the average distance of the k nearest samples of the minority class is the smallest. </li>
</ul>


In [ ]:

nm = NearMiss(random_state=42)
X_train_nm, y_train_nm = resampling_multiclass(pd.DataFrame(X_train_sm), pd.DataFrame(y_train_sm)[0], nm, ['detritus', 'feces'])
baseline_model(X_train_nm, y_train_nm, X_val, y_val, rf)['report']



In [ ]:

rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = resampling_multiclass(pd.DataFrame(X_train_sm), pd.DataFrame(y_train_sm)[0], rus, ['detritus', 'feces'])
baseline_model(X_train_rus, y_train_rus, X_val, y_val, rf)['report']




<p><strong>COMMENT.</strong></p>
<p>As can be seen from the results above, the undersampling techniques do not improve the f1-score macro, therefore, we skip this step and do not use it in the final models.</p>



<p><strong>FINALLY</strong>, we use the technique named <strong>TomekLinks</strong>, which is a <em>cleaning undersampling method</em>. The number of samples to be selected need not to be specified.</p>
<ul>
<li><em>TomekLinks</em>: it removes the samples considered noisy.</li>
</ul>


In [ ]:

# Undersampling using TomekLinks
tl = TomekLinks(sampling_strategy='all', random_state=42)
X_train_tl, y_train_tl = tl.fit_resample(X_train_sm, y_train_sm)
baseline_model(X_train_tl, y_train_tl, X_val, y_val, rf)['report']




<p><strong>COMMENT.</strong></p>
<p>The TomekLinks technique help to improve a bit of the score of model after resampling data. It increase the score from 54.38% (after using SMOTE) to 54.68% (after using TomekLinks).</p>



<p><strong>SUMMARY for RESAMPLING DATA</strong></p>
<p>Based on the result after resamplings in the training set and evaluate in the validation set, our training data will be <strong>oversample using SMOTE</strong> and <strong>then undersample using TomekLinks</strong>.</p>



<hr/>
<h3 id="3.-Model-Selection">3. Model Selection<a class="anchor-link" href="#3.-Model-Selection">¶</a></h3>



<p>We do not use cross-validation to measure the performance of models because the training set was resampled, which means when using cross-validation on the training set, the hold-out set was be seen and lead to over-fitting.</p>
<p>Insteads, we train models on using the training set (which was resampled) and evaluate them on the validation set (which is not resampled).</p>
<p>Here we use RandomForestClassifier and XGBClassifier for model selection.</p>


In [ ]:

rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf = baseline_model(X_train_tl, y_train_tl, X_val, y_val, rf)
model_rf['report']



In [ ]:

xgb = XGBClassifier(n_estimators=100, seed=42)
model_xgb = baseline_model(X_train_tl, y_train_tl, X_val.as_matrix(), y_val, xgb)
model_xgb['report']




<p><strong>COMMENT.</strong></p>
<p>Results:</p>
<ul>
<li><em>RandomForestClassifier</em>:<ul>
<li>F1-score macro: <strong>0.546899</strong></li>
<li>Runing time: <strong>500</strong> seconds</li>
</ul>
</li>
<li><em>XGBClassifier</em>:<ul>
<li>F1-score macro: <strong>0.450603</strong></li>
<li>Runing time: <strong>3734</strong> seconds</li>
</ul>
</li>
</ul>
<p><em>RandomForestClassifier</em> outperforms the <em>XGBClassifier</em> in both score and runing time. Random Forest is easier to implement in parallel.</p>
<p><strong>In conclusion, our final model is <em>RandomForestClassifier</em>.</strong></p>



<hr/>
<h3 id="4.-Parameter-Optimisation">4. Parameter Optimisation<a class="anchor-link" href="#4.-Parameter-Optimisation">¶</a></h3>



<p>The hyper-paremete tuning for RandomForestClassifier is:</p>
<p><code>{bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False
            }</code></p>



<hr/>
<h3 id="5.-Model-Evaluation">5. Model Evaluation<a class="anchor-link" href="#5.-Model-Evaluation">¶</a></h3>



<p>In this section, we evaluate the final model using the test set.</p>


In [ ]:

final_model = RandomForestClassifier(n_estimators=100, random_state=42)
final_result = baseline_model(X_train_tl, y_train_tl, X_test, y_test, final_model)
final_result['report']['macro avg']




<p>Our <strong>final evaluation</strong> f1-score avg=macro on a held-out test set is <strong>57.28%</strong>.</p>
<p>The time for training and prediction is 501 seconds.</p>
